# Enter State Farm

In [4]:
from theano.sandbox import cuda
cuda.use('gpu1')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [5]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"
path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [6]:
batch_size=64

## Create validation set

In [9]:
# point to your training images
train_dir = '/home/ubuntu/courses/deeplearning1/nbs/data/state/train'

# point to the 'driver_imgs_list.csv'
lookup = '/home/ubuntu/courses/deeplearning1/nbs/data/state/driver_imgs_list.csv'

# point to the validation directory, which will be created in the next block
val_dir = '/home/ubuntu/courses/deeplearning1/nbs/data/state/valid'

In [10]:
os.getcwd()

'/home/ubuntu/courses/deeplearning1/nbs'

In [12]:
# creates a directory, called 'validation', and into it, creates 10 subdirs, one for each class.
directory = '/home/ubuntu/courses/deeplearning1/nbs/data/state/valid'
if not os.path.exists(directory):
    os.makedirs(directory)
cwd = os.getcwd()
# path = cwd + '/' + directory
path = val_dir
[os.mkdir('{}/c{}'.format(path, i)) for i in range(10)]

[None, None, None, None, None, None, None, None, None, None]

In [13]:
# read in the 'driver_imgs_list.csv' file
df = pd.read_csv(lookup)

In [16]:
import random
??random.sample

In [17]:
# There are 26 unique drivers in the train set. 
# We want 20%, so pick 5 at random
driver_names = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
driver_names_for_validation = random.sample(driver_names, 5)

In [18]:
# filter df by selecting rows with the subjects we want
df = df[df['subject'].isin(driver_names_for_validation)]

In [23]:
import shutil

In [24]:
# do the move here:
i = 0
for index, row in df.iterrows():
    to_move = train_dir + '/' +  row['classname'] + '/' + row['img']
    print (to_move)
    i = i + 1
    move_to = val_dir + '/' +  row['classname']
    print (move_to)
    shutil.move(to_move, move_to)

/home/ubuntu/courses/deeplearning1/nbs/data/state/train/c0/img_51066.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/nbs/data/state/train/c0/img_19066.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/nbs/data/state/train/c0/img_13965.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/nbs/data/state/train/c0/img_12053.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/nbs/data/state/train/c0/img_23592.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/nbs/data/state/train/c0/img_57898.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/nbs/data/state/train/c0/img_47068.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/state/valid/c0
/home/ubuntu/courses/deeplearning1/nbs/data/state/train/c0/img_7276.jpg
/hom

In [27]:
# verify the result
print ('files to move: ' ,df.shape[0])
print ('files moved:   ' ,i)

files to move:  4696
files moved:    4696


## Create sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

In [28]:
%cd data/state

/home/ubuntu/courses/deeplearning1/nbs/data/state


In [29]:
%cd train

/home/ubuntu/courses/deeplearning1/nbs/data/state/train


In [30]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [31]:
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [32]:
from shutil import copyfile

In [33]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [34]:
%cd ../valid

/home/ubuntu/courses/deeplearning1/nbs/data/state/valid


In [35]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [36]:
%cd ../../..

/home/ubuntu/courses/deeplearning1/nbs


In [37]:
%mkdir data/state/results

In [38]:
%mkdir data/state/sample/test

## Create batches

In [9]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [10]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [41]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

As you can see below, this training is going nowhere...

In [42]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 13.6433 - acc: 0.1100 - val_loss: 14.4119 - val_acc: 0.1050
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 14.3259 - acc: 0.1107 - val_loss: 14.2541 - val_acc: 0.1140


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [43]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNormal(None, 3, 224, 224)   6           batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1505296
____________________________________________________________________________________________________


Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [44]:
10*3*224*224

1505280

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [46]:
np.round(model.predict_generator(batches, batches.N)[:10],2)

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]], dtype=float32)

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [47]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 31s - loss: 2.3907 - acc: 0.1827 - val_loss: 4.1721 - val_acc: 0.2180
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 1.7515 - acc: 0.4140 - val_loss: 3.1829 - val_acc: 0.2750


Great - we found our way out of that hole... Now we can increase the learning rate and see where we can get to.

In [48]:
model.optimizer.lr=0.001

In [49]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 32s - loss: 1.3895 - acc: 0.5953 - val_loss: 2.7577 - val_acc: 0.3050
Epoch 2/4
1500/1500 [==============================] - 24s - loss: 1.1520 - acc: 0.6833 - val_loss: 2.3568 - val_acc: 0.3270
Epoch 3/4
1500/1500 [==============================] - 25s - loss: 0.9870 - acc: 0.7667 - val_loss: 1.9178 - val_acc: 0.3420
Epoch 4/4
1500/1500 [==============================] - 25s - loss: 0.7941 - acc: 0.8467 - val_loss: 1.9685 - val_acc: 0.3370


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [50]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 1000 images belonging to 10 classes.


In [51]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

array([[ 1.97,  0.34],
       [ 1.96,  0.34],
       [ 1.95,  0.34],
       [ 1.95,  0.33],
       [ 2.02,  0.33],
       [ 1.95,  0.33],
       [ 1.95,  0.36],
       [ 1.97,  0.34],
       [ 1.97,  0.34],
       [ 2.  ,  0.34]])

Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [52]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 31s - loss: 2.5304 - acc: 0.2033 - val_loss: 6.5892 - val_acc: 0.1820
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 1.8878 - acc: 0.4507 - val_loss: 3.0630 - val_acc: 0.2500


In [53]:
model.optimizer.lr=0.001

In [54]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 32s - loss: 1.5423 - acc: 0.6147 - val_loss: 2.6242 - val_acc: 0.2900
Epoch 2/4
1500/1500 [==============================] - 25s - loss: 1.3075 - acc: 0.7233 - val_loss: 2.0126 - val_acc: 0.3220
Epoch 3/4
1500/1500 [==============================] - 25s - loss: 1.1259 - acc: 0.7913 - val_loss: 1.8647 - val_acc: 0.3510
Epoch 4/4
1500/1500 [==============================] - 24s - loss: 0.9848 - acc: 0.8340 - val_loss: 1.6592 - val_acc: 0.4370


Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [55]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 31s - loss: 1.9801 - acc: 0.3753 - val_loss: 7.9205 - val_acc: 0.1830
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 0.9485 - acc: 0.7420 - val_loss: 4.2055 - val_acc: 0.2550
Epoch 1/5
1500/1500 [==============================] - 32s - loss: 0.5275 - acc: 0.8880 - val_loss: 2.4043 - val_acc: 0.3350
Epoch 2/5
1500/1500 [==============================] - 25s - loss: 0.3198 - acc: 0.9620 - val_loss: 1.8213 - val_acc: 0.4720
Epoch 3/5
1500/1500 [==============================] - 24s - loss: 0.2096 - acc: 0.9820 - val_loss: 1.7280 - val_acc: 0.4470
Epoch 4/5
1500/1500 [==============================] - 24s - loss: 0.1394 - acc: 0.9940 - val_loss: 1.8370 - val_acc: 0.4060
Epoch 5/5
1500/1500 [==============================] - 25s - loss: 0.1086 - acc: 0.9973 - val_loss: 1.7935 - val_acc: 0.4140


Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [2]:
# it's a good idea to create a function if you need to run a piece of code multiple times!
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [57]:
conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 34s - loss: 1.6405 - acc: 0.5220 - val_loss: 2.2705 - val_acc: 0.1560
Epoch 2/2
1500/1500 [==============================] - 28s - loss: 0.3884 - acc: 0.9300 - val_loss: 2.1149 - val_acc: 0.1980
Epoch 1/4
1500/1500 [==============================] - 34s - loss: 0.1152 - acc: 0.9927 - val_loss: 2.2133 - val_acc: 0.1600
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.0625 - acc: 0.9953 - val_loss: 2.2775 - val_acc: 0.1800
Epoch 3/4
1500/1500 [==============================] - 29s - loss: 0.0286 - acc: 1.0000 - val_loss: 2.3266 - val_acc: 0.2320
Epoch 4/4
1500/1500 [==============================] - 28s - loss: 0.0187 - acc: 1.0000 - val_loss: 2.3398 - val_acc: 0.2400


The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [58]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [59]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 34s - loss: 2.1884 - acc: 0.3267 - val_loss: 2.1530 - val_acc: 0.1650
Epoch 2/2
1500/1500 [==============================] - 27s - loss: 1.1873 - acc: 0.6340 - val_loss: 2.1395 - val_acc: 0.1990
Epoch 1/4
1500/1500 [==============================] - 37s - loss: 0.7541 - acc: 0.7820 - val_loss: 2.1858 - val_acc: 0.2030
Epoch 2/4
1500/1500 [==============================] - 28s - loss: 0.5937 - acc: 0.8387 - val_loss: 2.5417 - val_acc: 0.1260
Epoch 3/4
1500/1500 [==============================] - 31s - loss: 0.4391 - acc: 0.8687 - val_loss: 2.6181 - val_acc: 0.1450
Epoch 4/4
1500/1500 [==============================] - 28s - loss: 0.3742 - acc: 0.9020 - val_loss: 2.6250 - val_acc: 0.1440


Height shift: move the image up and down -

In [65]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1568 images belonging to 10 classes.


In [66]:
model = conv1(batches)

Epoch 1/2
1568/1568 [==============================] - 11s - loss: 1.7843 - acc: 0.4458 - val_loss: 2.1259 - val_acc: 0.2375
Epoch 2/2
1568/1568 [==============================] - 11s - loss: 0.7028 - acc: 0.7825 - val_loss: 2.0232 - val_acc: 0.3164
Epoch 1/4
1568/1568 [==============================] - 11s - loss: 0.3586 - acc: 0.9152 - val_loss: 2.1772 - val_acc: 0.1806
Epoch 2/4
1568/1568 [==============================] - 11s - loss: 0.2335 - acc: 0.9490 - val_loss: 2.1935 - val_acc: 0.1727
Epoch 3/4
1568/1568 [==============================] - 11s - loss: 0.1626 - acc: 0.9656 - val_loss: 2.1944 - val_acc: 0.2106
Epoch 4/4
1568/1568 [==============================] - 11s - loss: 0.1214 - acc: 0.9758 - val_loss: 2.3481 - val_acc: 0.1766


Random shear angles (max in radians) -

In [67]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1568 images belonging to 10 classes.


In [68]:
model = conv1(batches)

Epoch 1/2
1568/1568 [==============================] - 11s - loss: 1.6148 - acc: 0.5223 - val_loss: 2.2513 - val_acc: 0.2475
Epoch 2/2
1568/1568 [==============================] - 11s - loss: 0.3915 - acc: 0.9203 - val_loss: 2.0757 - val_acc: 0.2725
Epoch 1/4
1568/1568 [==============================] - 11s - loss: 0.1478 - acc: 0.9821 - val_loss: 2.1869 - val_acc: 0.3084
Epoch 2/4
1568/1568 [==============================] - 11s - loss: 0.0831 - acc: 0.9904 - val_loss: 2.2449 - val_acc: 0.3164
Epoch 3/4
1568/1568 [==============================] - 11s - loss: 0.0530 - acc: 0.9955 - val_loss: 2.2426 - val_acc: 0.3154
Epoch 4/4
1568/1568 [==============================] - 11s - loss: 0.0343 - acc: 0.9994 - val_loss: 2.2609 - val_acc: 0.3234


Rotation: max in degrees -

In [69]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1568 images belonging to 10 classes.


In [70]:
model = conv1(batches)

Epoch 1/2
1568/1568 [==============================] - 11s - loss: 1.9734 - acc: 0.3865 - val_loss: 2.1849 - val_acc: 0.3064
Epoch 2/2
1568/1568 [==============================] - 11s - loss: 0.8523 - acc: 0.7411 - val_loss: 2.0310 - val_acc: 0.2655
Epoch 1/4
1568/1568 [==============================] - 11s - loss: 0.4652 - acc: 0.8833 - val_loss: 2.0401 - val_acc: 0.2036
Epoch 2/4
1568/1568 [==============================] - 11s - loss: 0.3448 - acc: 0.9101 - val_loss: 2.2149 - val_acc: 0.1317
Epoch 3/4
1568/1568 [==============================] - 11s - loss: 0.2411 - acc: 0.9420 - val_loss: 2.2614 - val_acc: 0.1287
Epoch 4/4
1568/1568 [==============================] - 11s - loss: 0.1722 - acc: 0.9636 - val_loss: 2.1208 - val_acc: 0.2106


Channel shift: randomly changing the R,G,B colors - 

In [76]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1568 images belonging to 10 classes.


In [77]:
model = conv1(batches)

Epoch 1/2
1568/1568 [==============================] - 11s - loss: 1.6381 - acc: 0.5191 - val_loss: 2.2146 - val_acc: 0.3483
Epoch 2/2
1568/1568 [==============================] - 11s - loss: 0.3530 - acc: 0.9305 - val_loss: 2.0966 - val_acc: 0.2665
Epoch 1/4
1568/1568 [==============================] - 11s - loss: 0.1036 - acc: 0.9923 - val_loss: 2.4195 - val_acc: 0.1766
Epoch 2/4
1568/1568 [==============================] - 11s - loss: 0.0450 - acc: 1.0000 - val_loss: 2.6192 - val_acc: 0.1667
Epoch 3/4
1568/1568 [==============================] - 11s - loss: 0.0259 - acc: 0.9994 - val_loss: 2.7227 - val_acc: 0.1816
Epoch 4/4
1568/1568 [==============================] - 11s - loss: 0.0180 - acc: 0.9994 - val_loss: 2.7049 - val_acc: 0.2206


And finally, putting it all together!

In [7]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [11]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 34s - loss: 1.5893 - acc: 0.5320 - val_loss: 2.2583 - val_acc: 0.2020
Epoch 2/2
1500/1500 [==============================] - 27s - loss: 0.3187 - acc: 0.9527 - val_loss: 1.9749 - val_acc: 0.2760
Epoch 1/4
1500/1500 [==============================] - 33s - loss: 0.0950 - acc: 0.9980 - val_loss: 2.0551 - val_acc: 0.2920
Epoch 2/4
1500/1500 [==============================] - 26s - loss: 0.0470 - acc: 0.9973 - val_loss: 2.1547 - val_acc: 0.2550
Epoch 3/4
1500/1500 [==============================] - 28s - loss: 0.0244 - acc: 1.0000 - val_loss: 2.2494 - val_acc: 0.2380
Epoch 4/4
1500/1500 [==============================] - 29s - loss: 0.0179 - acc: 1.0000 - val_loss: 2.3219 - val_acc: 0.2190


At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.

In [12]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1500/1500 [==============================] - 35s - loss: 0.0131 - acc: 1.0000 - val_loss: 2.3728 - val_acc: 0.1980
Epoch 2/5
1500/1500 [==============================] - 27s - loss: 0.0102 - acc: 1.0000 - val_loss: 2.3082 - val_acc: 0.2440
Epoch 3/5
1500/1500 [==============================] - 29s - loss: 0.0083 - acc: 1.0000 - val_loss: 2.2419 - val_acc: 0.2730
Epoch 4/5
1500/1500 [==============================] - 27s - loss: 0.0070 - acc: 1.0000 - val_loss: 2.1397 - val_acc: 0.3330
Epoch 5/5
1500/1500 [==============================] - 30s - loss: 0.0061 - acc: 1.0000 - val_loss: 2.0279 - val_acc: 0.3730


Lucky we tried that - we starting to make progress! Let's keep going.

In [13]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/25
1500/1500 [==============================] - 34s - loss: 0.0054 - acc: 1.0000 - val_loss: 1.9450 - val_acc: 0.3950
Epoch 2/25
1500/1500 [==============================] - 28s - loss: 0.0047 - acc: 1.0000 - val_loss: 1.8256 - val_acc: 0.4400
Epoch 3/25
1500/1500 [==============================] - 27s - loss: 0.0043 - acc: 1.0000 - val_loss: 1.7159 - val_acc: 0.4670
Epoch 4/25
1500/1500 [==============================] - 29s - loss: 0.0044 - acc: 1.0000 - val_loss: 1.6173 - val_acc: 0.4880
Epoch 5/25
1500/1500 [==============================] - 29s - loss: 0.0037 - acc: 1.0000 - val_loss: 1.5231 - val_acc: 0.5100
Epoch 6/25
1500/1500 [==============================] - 28s - loss: 0.0035 - acc: 1.0000 - val_loss: 1.4495 - val_acc: 0.5240
Epoch 7/25
1500/1500 [==============================] - 31s - loss: 0.0034 - acc: 1.0000 - val_loss: 1.3945 - val_acc: 0.5490
Epoch 8/25
1500/1500 [==============================] - 29s - loss: 0.0030 - acc: 1.0000 - val_loss: 1.3353 - val_acc:

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.